In [2]:
import globals
import os
import pandas as pd

# Loads confusion matrices from each CV, calculates the mean and standard deviation
# and saves them to a file

paths = [os.path.join(globals.BASE_PATH, "experiments", globals.CV_EXP_NAMES[fold], f"conf_matrix_cv_{fold}.csv") for fold in range(5)]

conf_matrices = [pd.read_csv(path, index_col=0) for path in paths]

mean_conf_matrix = pd.concat(conf_matrices).groupby(level=0).mean()
std_conf_matrix = pd.concat(conf_matrices).groupby(level=0).std()

mean_conf_matrix

    100% |████████████████████████████████| 9.5MB 80kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl
    100% |████████████████████████████████| 501kB 270kB/s ta 0:00:01
    100% |████████████████████████████████| 13.4MB 61kB/s eta 0:00:011
  Using cached https://files.pythonhosted.org/packages/d9/5a/e7c31adbe875f2abbb91bd84cf2dc52d792b5a01506781dbcf25c91daf11/six-1.16.0-py2.py3-none-any.whl


,benign,malignant,normal
benign,0.940271,0.036813,0.022917
malignant,0.056763,0.933934,0.009302
normal,0.089188,0.013793,0.897019


In [3]:
std_conf_matrix

,benign,malignant,normal
benign,0.070067,0.035187,0.036384
malignant,0.045362,0.062245,0.020801
normal,0.068689,0.030842,0.098632


In [ ]:
mean_conf_matrix.to_csv("mean_conf_matrix.csv")
std_conf_matrix.to_csv("std_conf_matrix.csv")